In [1]:
%reload_ext autoreload
%autoreload 2

import os, psutil

from rocket_fft import numpy_like, scipy_like

numpy_like()

os.environ['NUMEXPR_MAX_THREADS'] = '20'
os.environ['NUMBA_DEBUGINFO'] = '0'

from alphadia.extraction import processlogger
processlogger.init_logging()
import logging

logger = logging.getLogger()

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import neptune.new as neptune
import alphatims.bruker as bruker

#from alphabase.spectral_library.flat import SpecLibFlat
from alphabase.spectral_library.base import SpecLibBase
#from alphabase.spectral_library.reader import LibraryReaderBase

from alphadia.extraction.data import TimsTOFDIA
from alphadia.extraction.planning import Plan, Workflow
from alphadia.extraction.calibration import RunCalibration
from alphadia.extraction.candidateselection import MS1CentricCandidateSelection
from alphadia.extraction.scoring import fdr_correction, MS2ExtractionWorkflow
import alphadia.extraction.utils as utils
yaml_file = 'config.yaml'

raw_files = ['/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_syP5-25wo_06_S4-F9_1_656.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_syP5-25wo_05_S4-F6_1_653.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_syP5-25wo_04_S4-F3_1_650.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_syP5-25wo_03_S4-E11_1_647.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_AID8_10_S4-G9_1_667.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_AID8_09_S4-G6_1_664.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_AID8_08_S4-G3_1_661.d',
             '/Users/georgwallmann/Documents/data/alphadia_benchmarking/raw_data/2023_03_02_MCT_scp_hela/20230103_TIMS05_MCT_SA_HeLa_SM03p7b_Ref10_AID8_07_S4-F11_1_658.d'
            ]

output_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/alphadia_runs'

try:
    neptune_token = os.environ['NEPTUNE_TOKEN']
except KeyError:
    logger.error('NEPTUNE_TOKEN environtment variable not set')


0:00:01.012684  WARNING: WARNING: Temp mmap arrays are written to /var/folders/lc/9594t94d5b5_gn0y04w1jh980000gn/T/temp_mmap_i018pmny. Cleanup of this folder is OS dependant, and might need to be triggered manually! Current space: 1,163,079,553,024
0:00:01.023988  WARNING: WARNING: No Bruker libraries are available for this operating system. Mobility and m/z values need to be estimated. While this estimation often returns acceptable results with errors < 0.02 Th, huge errors (e.g. offsets of 6 Th) have already been observed for some samples!
0:00:01.024318  INFO: 


In [2]:
test_lib = SpecLibBase()
test_lib_location = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/marvin_scp/MSfragger_library_mod_noLossType_d0_shared_eg_n_fragments.hdf'
test_lib.load_hdf(test_lib_location, load_mod_seq=True)

test_lib.precursor_df = test_lib.precursor_df[test_lib.precursor_df['channel'] == 0].copy()
#test_lib.precursor_df.elution_group_idx = np.arange(len(test_lib.precursor_df))

In [3]:
plan = Plan(raw_files[5:6])
plan.from_spec_lib_base(test_lib)
#plan.run(output_location, keep_decoys=True, fdr=1.0, neptune_token=neptune_token, neptune_tags='scp')

for dia_data, precursors_flat, fragments_flat in plan.get_run_data():
    break

0:00:10.610186  PROGRESS:       _   _      _         ___ ___   _   
0:00:10.610723  PROGRESS:      /_\ | |_ __| |_  __ _|   \_ _| /_\  
0:00:10.611034  PROGRESS:     / _ \| | '_ \ ' \/ _` | |) | | / _ \ 
0:00:10.611465  PROGRESS:    /_/ \_\_| .__/_||_\__,_|___/___/_/ \_\
0:00:10.611786  PROGRESS:            |_|                            
0:00:10.612025  PROGRESS: 
0:00:10.612509  INFO: loading default config from /Users/georgwallmann/Documents/git/alphadia/alphadia/extraction/../../misc/config/default.yaml
0:00:10.618095  PROGRESS: version: 1.0.1
0:00:10.618489  PROGRESS: hostname: Georgs-MacBook-Pro.local
0:00:10.618737  PROGRESS: date: 2023-04-11 19:15:20
0:00:14.287594  INFO: renaming precursor_columns columns
0:00:14.288536  INFO: renaming fragment_columns columns
0:00:14.288970  INFO: ========= Library Stats =========
0:00:14.289389  INFO: Number of precursors: 91,732
0:00:14.302115  INFO: 	thereof targets:45,867
0:00:14.302573  INFO: 	thereof decoys: 45,865
0:00:14.303756  INFO:

In [4]:
jit_data = dia_data.jitclass(transpose=True)

In [5]:
fragments_flat = pd.read_csv(os.path.join('/Users/georgwallmann/Documents/data/alphadia_benchmarking/alphadia_runs/2023_04_07_v1.0.2/fragments_flat_r.tsv'), sep='\t')
precursors_flat = pd.read_csv(os.path.join('/Users/georgwallmann/Documents/data/alphadia_benchmarking/alphadia_runs/2023_04_07_v1.0.2/precursors_flat_r.tsv'), sep='\t')

In [35]:
from alphadia.extraction import hybridselection

selection = hybridselection.HybridCandidateSelection(
    dia_data,
    precursors_flat,
    fragments_flat,
    rt_tolerance=30,
    mz_tolerance=15,
    mobility_tolerance=0.03,
    candidate_count=5,
    thread_count=10,
    debug=True,
    group_channels=False,
    top_k_precursors=3,
    top_k_fragments=12,
    rt_column='rt_calibrated',
    precursor_mz_column='mz_calibrated',
    fragment_mz_column='mz_library',
    mobility_column='mobility_calibrated'
)
egs = selection()

0:34:27.533204  INFO: Duty cycle consists of 9 frames, 0.96 seconds cycle time
0:34:27.533635  INFO: Duty cycle consists of 928 scans, 0.00065 1/K_0 resolution
0:34:27.533872  INFO: Expected peak length in RT is 10.00 seconds, sigma is 5.21
0:34:27.534057  INFO: Expected peak length in mobility is 0.03 1/K_0, sigma is 23.20
0:34:27.661668  INFO: starting candidate selection


{'rt_tolerance': 30, 'mobility_tolerance': 0.03, 'mz_tolerance': 15, 'candidate_count': 5, 'rt_column': 'rt_calibrated', 'mobility_column': 'mobility_calibrated', 'precursor_mz_column': 'mz_calibrated', 'fragment_mz_column': 'mz_library', 'thread_count': 10, 'debug': True, 'group_channels': False, 'exclude_shared_fragments': True, 'top_k_fragments': 12, 'top_k_precursors': 3}


100%|██████████| 10/10 [00:00<00:00, 812.27it/s]


In [34]:
import numba as nb

@nb.njit
def calculate_dia_cycle_mask(
        cycle : np.ndarray,
        quad_slices : np.ndarray
    ):

    """ Calculate the DIA cycle quadrupole mask for each score group.

    Parameters
    ----------

    cycle : np.ndarray
        The DIA mz cycle as part of the bruker.TimsTOF object. (n_frames * n_scans)

    quad_slices : np.ndarray
        The quadrupole slices for each score group. (n_score_groups, 2)

    Returns
    -------

    np.ndarray
        The DIA cycle quadrupole mask for each score group. (n_score_groups, n_frames * n_scans)
    """

    n_score_groups = quad_slices.shape[0]

    dia_mz_cycle = cycle.reshape(-1, 2)

    mz_mask = np.zeros((n_score_groups, len(dia_mz_cycle)), dtype=np.bool_)
    for i, (mz_start, mz_stop) in enumerate(dia_mz_cycle):
        for j, (quad_mz_start, quad_mz_stop) in enumerate(quad_slices):
            if (quad_mz_start <= mz_stop) and (quad_mz_stop >= mz_start):
                mz_mask[j, i] = True

    return mz_mask

In [16]:
for eg in egs[:10]:
    #print(eg.score_group_precursor_mz)
    #print(eg.score_group_fragment_mz)

    quadrupole_mz = hybridselection.calculate_score_group_limits(
        eg.score_group_precursor_mz, 
        eg.score_group_precursor_intensity
    )

    mz_mask = calculate_dia_cycle_mask(
        jit_data.cycle,
        

    )

    print(quadrupole_mz)

[[449.5776062 450.246521 ]]
[[449.24584961 449.9147644 ]]
[[529.25439453 529.92327881]]
[[519.58728027 520.25616455]]
[[420.5463562 421.215271 ]]
[[429.89013672 430.55905151]]
[[361.88458252 362.21902466]]
[[352.88928223 353.22372437]]
[[495.89105225 496.55996704]]
[[496.56292725 497.23184204]]


In [31]:
def test_calculate_score_group_limits():
    for f in [hybridselection.calculate_score_group_limits, hybridselection.calculate_score_group_limits.py_func]:
        for i in range(1000):
            mz_values = np.random.random(100)*1000
            intensity_values = np.floor(np.random.random(100) * 2)
            
            min = mz_values[intensity_values != 0.].min()
            max = mz_values[intensity_values != 0.].max()

            limits = hybridselection.calculate_score_group_limits(
                mz_values,
                intensity_values
            )

            assert limits.shape == (1,2)
            assert np.allclose(limits[0,0] , min)
            assert np.allclose(limits[0,1] , max)


test_calculate_score_group_limits()

In [7]:
scoring. egs

,elution_group_idx,score_group_idx,precursor_idx,rank,score,precursor_mz,decoy,scan_center,scan_start,scan_stop,...,mobility_calibrated,flat_frag_start_idx,flat_frag_stop_idx,charge,i_0,i_1,i_2,i_3,i_4,i_5
0,0,0,25488,0,25.837534,449.577606,0,836,826,847,...,0.753400,0,10,3,0.486296,0.330465,0.132628,0.039262,0.009423,0.001926
1,0,0,25488,1,23.586605,449.577606,0,872,846,880,...,0.753400,0,10,3,0.486296,0.330465,0.132628,0.039262,0.009423,0.001926
2,0,0,25488,2,20.277398,449.577606,0,876,860,880,...,0.753400,0,10,3,0.486296,0.330465,0.132628,0.039262,0.009423,0.001926
3,0,0,25488,3,2.612197,449.577606,0,808,800,819,...,0.753400,0,10,3,0.486296,0.330465,0.132628,0.039262,0.009423,0.001926
4,0,0,25488,4,0.922434,449.577606,0,870,860,880,...,0.753400,0,10,3,0.486296,0.330465,0.132628,0.039262,0.009423,0.001926
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413175,45865,91729,84142,3,10.129126,704.391052,1,465,453,478,...,0.992485,2779354,2779371,3,0.310872,0.344713,0.209841,0.091405,0.031640,0.009202
413176,45866,91730,77759,0,14.091162,1076.564209,0,123,112,135,...,1.234279,545157,545165,2,0.258218,0.303294,0.225582,0.125217,0.056558,0.021699
413177,45866,91730,77759,1,13.933697,1076.564209,0,59,56,71,...,1.234279,545157,545165,2,0.258218,0.303294,0.225582,0.125217,0.056558,0.021699
413178,45866,91730,77759,2,11.888897,1076.564209,0,131,113,136,...,1.234279,545157,545165,2,0.258218,0.303294,0.225582,0.125217,0.056558,0.021699


In [10]:
egs.columns

Index(['elution_group_idx', 'score_group_idx', 'precursor_idx', 'rank',
       'score', 'precursor_mz', 'decoy', 'scan_center', 'scan_start',
       'scan_stop', 'frame_center', 'frame_start', 'frame_stop', 'rt_library',
       'rt_calibrated', 'mobility_library', 'mobility_calibrated',
       'flat_frag_start_idx', 'flat_frag_stop_idx', 'charge', 'i_0', 'i_1',
       'i_2', 'i_3', 'i_4', 'i_5'],
      dtype='object')

In [8]:
from alphadia.extraction import quadrupole
q = quadrupole.SimpleQuadrupole(dia_data.cycle)

In [9]:
egs['mz'] = egs['precursor_mz']


In [ ]:
from alphadia.extraction.scoring import MS2ExtractionWorkflow

workflow = MS2ExtractionWorkflow(
    dia_data,
    precursors_flat,
    fragments_flat,
    egs,
    q,
    precursor_mz_tolerance=100,
    fragment_mz_tolerance=100,
    debug=False
)
psm = workflow()

In [ ]:
precursor, fragment = psm

In [ ]:
from alphadia.extraction.scoring import fdr_correction

In [ ]:
fdr_correction(precursor)

In [ ]:
df['mz'] = df['mz'].astype(np.float32)

In [ ]:
df['mz'].dtype

In [ ]:
precursors_flat['rt_library'].hist(bins=100)

In [ ]:
start = []
stop = []
for eg in egs[:10]:
    print(eg.rt)
    #start.append(eg.frame_limits[0,0])
    #stop.append(eg.frame_limits[0,1])

start = np.array(start)
stop = np.array(stop)

In [ ]:
features = []
decoy = []
rank = []
for eg in egs:
    for c in eg.candidates:
        features.append(c.features)
        decoy.append(c.decoy)
        rank.append(c.rank)

features = np.array(features)
decoy = np.array(decoy)

In [ ]:
egs[0].candidates[0].precursor_mz

In [ ]:
features[:,1] = np.log(features[:,1])
features[:,3] = np.log(features[:,3])
features[:,4] = np.log(features[:,4])
features[:,6] = np.log(features[:,6])

In [ ]:
for i in range(features.shape[1]):
    plt.hist(features[decoy==0, i], bins=100)
    plt.hist(features[decoy==1, i], bins=100)
    plt.show()

In [ ]:
# perform standard scaling, perform a logistic regression and plot the results
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

scaler = StandardScaler()
scaled_features = scaler.fit_transform(features)

lr = LogisticRegression()
lr.fit(scaled_features, decoy)


from sklearn.metrics import roc_curve, auc

lr.predict_proba(scaled_features).shape

fpr, tpr, _ = roc_curve(decoy, lr.predict_proba(scaled_features)[:, 1])
roc_auc = auc(fpr, tpr)

plt.figure()
lw = 2
plt.plot(fpr, tpr, color='darkorange',
            lw=lw, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver operating characteristic example')
plt.legend(loc="lower right")



In [ ]:
# iterate columns of scaled_features and plot the histogram by the decoy status
for i in range(features.shape[1]):
    plt.figure()
    plt.hist(features[decoy, i], bins=100, alpha=0.5, label='decoy')
    plt.hist(features[~np.array(decoy), i], bins=100, alpha=0.5, label='target')
    plt.legend()
    plt.show()

In [ ]:
scaled_features.shape

In [ ]:
from alphadia.extraction.numba import numeric

In [ ]:
from alphadia.extraction.hybridselection import GaussianFilter
gaussian_filter = GaussianFilter(
    jit_data
)
kernel = gaussian_filter.get_kernel()

In [ ]:
from alphadia.extraction import hybridselection
eg = egs[22]
print(eg.precursor_decoy)
features = hybridselection.build_candidates(
    eg.dense_precursors,
    eg.dense_fragments,
    eg.score_group_precursor_intensity,
    eg.score_group_fragment_intensity,
    eg,
    kernel,
    jit_data,
    candidate_count = 3,
    debug=True,
    weights = np.array([1,1,1,1,1,0.5,0.5,0.5]),
)

In [ ]:
len(features)

In [ ]:
feature_vector = []
for f in features:
    feature_vector.append(f.features)

feature_vector = np.array(feature_vector)

In [ ]:
eg.score_group_precursor_mz

In [ ]:
jit_data.scan_max_index

In [ ]:
eg.score_group_fragment_intensity

In [ ]:
n_scores = eg.precursor_intensity.shape[0]

for score_idx in range(n_scores):

    fragment_mask = fragment_intensity[score_idx] > 0
    precursor_mask = precursor_intensity[score_idx] > 0


In [ ]:
for i in range(s.shape[0]):
    plt.imshow(s[i])
    plt.show()

In [ ]:
for eg in egs:

    if eg.dense_precursors.shape[-1] > 10:
        s = build_score(
            eg.dense_precursors, 
            eg.dense_fragments, 
            eg.precursor_mz,
            eg.precursor_intensity,
            eg.fragment_mz,
            eg.fragment_intensity,
            kernel
        )

In [ ]:
egs[0].fragment_intensity
egs[0].decoy

In [ ]:
plt.imshow(s[0])

In [ ]:
from alphadia.extraction.numba import numeric
from alphadia.extraction.numba import fragments
import numba as nb
from tqdm import tqdm

In [ ]:
#%%timeit

tof_slice, values, push_indices, absolute_precursor_index, tof_indices = filter_push(
    precursor_tof_limits,
    jit_data.push_indices,
    jit_data.tof_indptr, 
    push_query,
    _absolute_precursor_index
)

precursor_index = np.unique(absolute_precursor_index)
n_precursor_indices = len(precursor_index)
precursor_index_reverse = np.zeros(np.max(precursor_index)+1, dtype=np.int64)
precursor_index_reverse[precursor_index] = np.arange(len(precursor_index))

relative_precursor_index = precursor_index_reverse[absolute_precursor_index]

frame_indices = push_indices // jit_data.scan_max_index
scan_indices = push_indices % jit_data.scan_max_index
precursor_cycle_indices = (frame_indices-jit_data.zeroth_frame)//jit_data.cycle.shape[1]

# cycle values
precursor_cycle_start = int(eg.frame_limits[0,0]-jit_data.zeroth_frame)//jit_data.cycle.shape[1]
precursor_cycle_stop = int(eg.frame_limits[0,1]-jit_data.zeroth_frame)//jit_data.cycle.shape[1]
precursor_cycle_len = precursor_cycle_stop - precursor_cycle_start

# scan valuesa
mobility_start = eg.scan_limits[0,0]
mobility_stop = eg.scan_limits[0,1]
mobility_len = mobility_stop - mobility_start

#tof_indices = jit_data.tof_indices[values]
mz_values = jit_data.mz_values[tof_indices]

intensities = jit_data.intensity_values_t[values]

# number of channels: intensity, mz
n_channels = 2

n_tof_slices = len(precursor_tof_limits)

dense_output = np.zeros(
    (
        n_channels, 
        n_tof_slices,
        n_precursor_indices,
        mobility_len,
        precursor_cycle_len
    ), 
    dtype=np.float64
)

# create dense intensities
for i, (_tof_slice, intensity, mz, p_slice) in enumerate(zip(tof_slice, intensities, mz_values, relative_precursor_index)):
    mobility = scan_indices[i]-mobility_start
    precursor_cycle = precursor_cycle_indices[i]-precursor_cycle_start
    dense_output[0, _tof_slice, p_slice ,mobility, precursor_cycle] += intensity

for i in range(n_tof_slices):
    plt.imshow(dense_output[0,i,0,:,:])
    plt.show()


In [ ]:
values.shape

In [ ]:
np.unique(absolute_precursor_index)

In [ ]:
push_indices.shape

In [ ]:
tof_slice.shape

In [ ]:
nb.typeof(jit_data.push_indices)

In [ ]:
push_query.shape

In [ ]:
push_indices.shape

In [ ]:
absolute_precursor_index.shape

In [ ]:
push_indices, absolute_precursor_cycle = get_push_indices(
        jit_data,
        eg.frame_limits,
        eg.scan_limits,
        fragment_cycle_mask
    )

In [ ]:
push_indices.shape

In [ ]:
np.unique(absolute_precursor_cycle)

In [ ]:
np.unique(source_indices)

In [ ]:
np.unique(absolute_precursor_index)

In [ ]:
quadrupole_mz

In [ ]:
push_query.shape

In [ ]:
cycle_limits = eg.frame_limits//jit_data.cycle.shape[1]

In [ ]:
@nb.njit

def get_optimized_push_indices(
        jit_data,
        cycle_limits,
        scan_limits,
        score_group_cycle_mask,
        precursor_cycle_mask,
    ):

    for cycle_index in range(cycle_limits[0,0], cycle_limits[0,1]):
        cycle_start_index = cycle_index * jit_data.cycle.shape[1] * jit_data.scan_max_index

        
get_optimized_push_indices(
        jit_data,
        cycle_limits,
        eg.scan_limits,
        score_group_cycle_mask,
        precursor_cycle_mask,
    )

In [ ]:
cycle_limits

In [ ]:



             
push_indices, score_group, absolute_precursor_cycle = get_push_indices(
    jit_data,
    eg.frame_limits,
    eg.scan_limits,
    score_group_cycle_mask,
    precursor_cycle_mask,
)


In [ ]:
%timeit get_push_indices(jit_data, eg.frame_limits, eg.scan_limits, score_group_cycle_mask, precursor_cycle_mask)

In [ ]:
@nb.njit
def unique(a):
    return np.array(list(set(a)))

%timeit unique(absolute_precursor_cycle)

In [ ]:
precursor_mz

In [ ]:
print()

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(10,5))
pos = ax.imshow(fragments_dense[1,1,0,:,:],cmap='RdBu_r')
fig.colorbar(pos, ax=ax)
plt.show()

In [ ]:
%%timeit
get_dense_hybrid(
    jit_data, 
    eg.frame_limits,
    eg.scan_limits,
    precursor_mz,
    120,
    fragment_mz,
    120,
    quadrupole_mz
    )

In [ ]:
fragment_mz

In [ ]:
%timeit get_dense_hybrid( jit_data, eg.frame_limits, eg.scan_limits, precursor_mz, fragment_mz, quadrupole_mz)

In [ ]:
import numba as nb

os.environ['DEBUG'] = 'True'

from alphadia.extraction.hybridselection import HybridElutionGroup, HybridElutionGroupContainer

def assemble_elution_groups(
        precursors_flat,
        rt_column='rt_library',
        mobility_column='mobility_library',
        precursor_mz_column='mz_library'
        ):
    
    """
    Assemble elution groups from precursor library.

    Parameters
    ----------

    precursors_flat : pandas.DataFrame
        Precursor library.

    rt_column : str
        Column name of the retention time values.

    mobility_column : str
        Column name of the mobility values.

    precursor_mz_column : str
        Column name of the precursor m/z values.

    Returns
    -------
    HybridElutionGroupContainer
        Numba jitclass with list of elution groups.
    """
    
    if len(precursors_flat) == 0:
        return

    available_isotopes = utils.get_isotope_columns(precursors_flat.columns)
    available_isotope_columns = [f'i_{i}' for i in available_isotopes]

    precursors_sorted = precursors_flat.sort_values('elution_group_idx').copy()

    @nb.njit(debug=True)
    def assemble_njit(
        elution_group_idx,
        precursor_idx,
        channel,
        flat_frag_start_stop_idx,
        rt_values,
        mobility_values,
        charge,
        decoy,
        precursor_mz,
        isotope_intensity
    ):
        elution_group = elution_group_idx[0]
        elution_group_start = 0
        elution_group_stop = 0

        eg_list = []
        
        while elution_group_stop < len(elution_group_idx)-1:
            
            elution_group_stop += 1

            if elution_group_idx[elution_group_stop] != elution_group:
                    
                eg_list.append(HybridElutionGroup(    
                    elution_group,
                    precursor_idx[elution_group_start:elution_group_stop],
                    channel[elution_group_start:elution_group_stop],
                    flat_frag_start_stop_idx[elution_group_start:elution_group_stop],
                    rt_values[elution_group_start],
                    mobility_values[elution_group_start],
                    charge[elution_group_start],
                    decoy[elution_group_start:elution_group_stop],
                    precursor_mz[elution_group_start:elution_group_stop],
                    isotope_intensity[elution_group_start:elution_group_stop]
                ))

                elution_group_start = elution_group_stop
                elution_group = elution_group_idx[elution_group_start]
                
        egs = nb.typed.List(eg_list)
        return HybridElutionGroupContainer(egs)

    return assemble_njit(
        precursors_sorted['elution_group_idx'].values.astype(np.uint32),
        precursors_sorted['precursor_idx'].values.astype(np.uint32),
        precursors_sorted['channel'].values.astype(np.uint32),
        precursors_sorted[['flat_frag_start_idx','flat_frag_stop_idx']].values.copy().astype(np.uint32),
        precursors_sorted[rt_column].values.astype(np.float64),
        precursors_sorted[mobility_column].values.astype(np.float64),
        precursors_sorted['charge'].values.astype(np.uint8),
        precursors_sorted['decoy'].values.astype(np.uint8),
        precursors_sorted[precursor_mz_column].values.astype(np.float64),
        precursors_sorted[available_isotope_columns].values.copy().astype(np.float32),
    )
    

In [ ]:
assemble_elution_groups(precursors_flat)

In [ ]:
%timeit assemble_elution_groups(precursors_flat)

In [ ]:
from alphadia.extraction import candidateselection

gaussian = candidateselection.GaussianFilter(dia_data)
kernel = gaussian.get_kernel()

In [ ]:
plt.imshow(kernel)

In [ ]:
kernel[:,10:].shape

In [ ]:
p = dense_precursors[0,0,:,:32,:64]
p.shape

In [ ]:
x = np.tile(p, (10,1,1,1))

In [ ]:
x.shape

In [ ]:
%timeit smooth = fourier_a1(x, kernel)

In [ ]:
plt.imshow(p[0])

In [ ]:
plt.imshow(smooth[0])

In [ ]:
plt.imshow(roll(smooth[0,0],-10,-10))

In [ ]:
import numba as nb
import numpy as np

@nb.njit
def jit_fft(x):
    return np.fft.fft(x)

a = np.array([2, 7, 1, 8, 2, 8, 1, 8])
jit_fft(a)

In [ ]:
from alphadia.extraction import quadrupole, scoring, features
q = quadrupole.SimpleQuadrupole(dia_data.cycle)

In [ ]:
from alphadia.extraction import scoring

extraction = scoring.MS2ExtractionWorkflow(
    dia_data,
    precursors_flat,
    fragments_flat,
    candidates,
    q,
    precursor_mz_tolerance=30,
    fragment_mz_tolerance=30
)

feature_df, fragment_df = extraction()

In [ ]:
df = scoring.fdr_correction(feature_df)

In [ ]:
df = scoring.fdr_correction(df)